# Setup

In [1]:
import pandas as pd
import numpy as np
import os

# Program

## EDA

In [3]:
one_year_data = pd.read_csv("../data/final/oneyear_data.csv")

In [4]:
one_year_data.describe()

,YEAR,MO,DY,HR,T2M,WS2M,RH2M,PRECTOTCORR,LAT,LONG
count,2.889936e+07,2.889936e+07,2.889936e+07,2.889936e+07,2.889936e+07,2.889936e+07,2.889936e+07,2.889936e+07,2.889936e+07,2.889936e+07
mean,2.023754e+03,6.513661e+00,1.575683e+01,1.150000e+01,1.936495e+01,-4.928667e+00,7.360668e+01,-7.927540e+00,-2.662500e+00,1.176595e+02
std,4.306205e-01,3.451234e+00,8.811530e+00,6.922187e+00,9.259959e+01,9.039623e+01,9.778027e+01,9.010100e+01,5.049753e+00,1.356696e+01
min,2.023000e+03,1.000000e+00,1.000000e+00,0.000000e+00,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-1.116250e+01,9.440950e+01
25%,2.024000e+03,4.000000e+00,8.000000e+00,5.750000e+00,2.708000e+01,1.460000e+00,7.831000e+01,2.000000e-02,-7.162500e+00,1.059095e+02
50%,2.024000e+03,7.000000e+00,1.600000e+01,1.150000e+01,2.827000e+01,3.090000e+00,8.194000e+01,9.000000e-02,-2.662500e+00,1.176595e+02
75%,2.024000e+03,1.000000e+01,2.300000e+01,1.725000e+01,2.897000e+01,4.860000e+00,8.575000e+01,2.500000e-01,1.837500e+00,1.294095e+02
max,2.024000e+03,1.200000e+01,3.100000e+01,2.300000e+01,4.247000e+01,1.856000e+01,1.000000e+02,1.888500e+02,5.837500e+00,1.409095e+02


In [5]:
one_year_data.rename(columns={'YEAR':'Year', "MO":"Month", "DY":"Day", "HR":"Hour"}, inplace=True)

In [6]:
one_year_data

,Year,Month,Day,Hour,T2M,WS2M,RH2M,PRECTOTCORR,LAT,LONG
0,2023,10,3,0,27.31,6.12,75.62,0.01,-0.1625,94.4095
1,2023,10,3,1,27.23,5.98,75.44,0.01,-0.1625,94.4095
2,2023,10,3,2,27.18,5.88,75.12,0.00,-0.1625,94.4095
3,2023,10,3,3,27.15,5.99,75.19,0.01,-0.1625,94.4095
4,2023,10,3,4,27.15,6.23,75.38,0.01,-0.1625,94.4095
...,...,...,...,...,...,...,...,...,...,...
28899355,2024,10,2,19,29.15,1.57,74.50,0.02,5.8375,140.9095
28899356,2024,10,2,20,29.16,1.80,74.94,0.01,5.8375,140.9095
28899357,2024,10,2,21,29.25,2.20,75.56,0.03,5.8375,140.9095
28899358,2024,10,2,22,29.25,2.59,76.44,0.02,5.8375,140.9095


In [7]:
one_year_data["DATETIME"] = pd.to_datetime(one_year_data[["Year", "Month", "Day", "Hour"]])

In [8]:
one_year_data.sort_values(by="DATETIME", inplace=True)

## Split Data

In [9]:
pivot = round(0.9*len(one_year_data))

In [10]:
train_data = one_year_data.iloc[:pivot]
test_data = one_year_data.iloc[pivot:]

## Modeling

In [11]:
from xgboost import XGBRegressor
import xgboost as xgb
import pandas as pd
# from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [12]:
X_test = test_data.drop(columns=["DATETIME", "T2M", "WS2M", "RH2M", "PRECTOTCORR"])
y_test = test_data["T2M"]

In [13]:
X_train = train_data.drop(columns=["DATETIME", "T2M", "WS2M", "RH2M", "PRECTOTCORR"])
y_train = train_data["T2M"]

In [14]:
model = XGBRegressor(n_estimators=100)

In [15]:
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [16]:
y_hat = model.predict(X_test)

In [17]:
mae = mean_absolute_error(y_test, y_hat)
print(f'Mean Absolute Error: {mae:.2f}')

Mean Absolute Error: 1.14


In [97]:
data_sample =  np.array([[2023, 11, 7, 4, -6.914744, 107.609810]]) #bandung

In [98]:
# Make predictions using the trained model
predictions = model.predict(data_sample)

# Print the predictions
print("Predictions for the new data:")
print(predictions)


Predictions for the new data:
[25.0566]


In [361]:
model.save_model('../model/temperature_model.json')

### Windspeed model

In [302]:
windspeed_model = XGBRegressor(n_estimators=100)

In [303]:
X_train = train_data.drop(columns=["DATETIME", "WS2M"])
y_train = train_data["WS2M"]

In [304]:
X_test = test_data.drop(columns=["DATETIME", "WS2M"])
y_test = test_data["WS2M"]   

In [305]:
windspeed_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [306]:
y_hat = windspeed_model.predict(X_test)

In [307]:
mae = mean_absolute_error(y_test, y_hat)
print(f'Mean Absolute Error: {mae:.2f}')

Mean Absolute Error: 0.88


In [308]:
data_sample =  np.array([[2024, 10, 7, 13, 31.0 , 49.0, 0, -6.914744, 107.609810]]) # bandung

In [309]:
# Make predictions using the trained model
predictions = windspeed_model.predict(data_sample)

# Print the predictions
print("Predictions for the new data:")
print(predictions)


Predictions for the new data:
[3.176307]


In [310]:
windspeed_model.save_model('../model/windspeed_model.json')

### Humidity Model

In [386]:
humidity_model = XGBRegressor(n_estimators=100)

In [387]:
X_train = train_data.drop(columns=["DATETIME", "RH2M"])
y_train = train_data["RH2M"]

In [388]:
X_test = test_data.drop(columns=["DATETIME", "RH2M"])
y_test = test_data["RH2M"]   

In [389]:
humidity_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [390]:
y_hat = humidity_model.predict(X_test)

In [391]:
mae = mean_absolute_error(y_test, y_hat)
print(f'Mean Absolute Error: {mae:.2f}')

Mean Absolute Error: 1.62


In [392]:
data_sample =  np.array([[2024, 10, 7, 13, 31.0 , 3.61, 0, -6.914744, 107.609810]]) # bandung

In [393]:
# Make predictions using the trained model
predictions = humidity_model.predict(data_sample)

# Print the predictions
print("Predictions for the new data:")
print(predictions)


Predictions for the new data:
[56.728752]


In [397]:
humidity_model.save_model('../model/humidity_model.json')

### Precipitation Model

In [95]:
precipitation_model = XGBRegressor()

In [96]:
X_train = train_data.drop(columns=["DATETIME", "PRECTOTCORR"])
y_train = train_data["PRECTOTCORR"]

In [97]:
X_test = test_data.drop(columns=["DATETIME", "PRECTOTCORR"])
y_test = test_data["PRECTOTCORR"]   

In [98]:
precipitation_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [99]:
y_hat = precipitation_model.predict(X_test)

In [100]:
mae = mean_absolute_error(y_test, y_hat)
print(f'Mean Absolute Error: {mae:.2f}')

Mean Absolute Error: 0.29


In [394]:
data_sample =  np.array([[2024, 10, 7, 13, 31.0 , 3.61, 49.0, -6.914744, 107.609810]]) # bandung

In [395]:
# Make predictions using the trained model
predictions = precipitation_model.predict(data_sample)

# Print the predictions
print("Predictions for the new data:")
print(predictions)


Predictions for the new data:
[0.07967559]


In [396]:
precipitation_model.save_model('../model/precipitation_model.json')